In [ ]:
#following blocks of code for transformations and pipeline come from 
#https://github.com/ageron/handson-ml2
#03_classification

from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [ ]:
#Find most frequent entry and fill NA 
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        #
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["CreditScore","Age", "Tenure", "Balance", "EstimatedSalary"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

In [ ]:
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Gender", "Geography","NumOfProducts", "HasCrCard", "IsActiveMember"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [ ]:
#put together the two pipelines
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("cat_pipeline", cat_pipeline),
        ("num_pipeline", num_pipeline)
    ])
   
    

In [ ]:
X_train = preprocess_pipeline.fit_transform(X_train)